# Solar flares / AR classification

In [90]:
import keras 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import requests 
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras.preprocessing.image import load_img
from keras.preprocessing.image import save_img
from keras.preprocessing.image import img_to_array
import os
import numpy as np
from PIL import Image
import random
import math
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]
from keras.optimizers import SGD

In [91]:
from selenium import webdriver
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from nltk.tokenize import sent_tokenize, word_tokenize 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
from ann_visualizer.visualize import ann_viz

In [92]:
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors 
from matplotlib import cm
import math 
plt.style.use('fivethirtyeight')
plt.rcParams['font.family'] = 'sans-serif' 
plt.rcParams['font.serif'] = 'Ubuntu' 
plt.rcParams['font.monospace'] = 'Ubuntu Mono' 
plt.rcParams['font.size'] = 14 
plt.rcParams['axes.labelsize'] = 12 
plt.rcParams['axes.labelweight'] = 'bold' 
plt.rcParams['axes.titlesize'] = 12 
plt.rcParams['xtick.labelsize'] = 12 
plt.rcParams['ytick.labelsize'] = 12 
plt.rcParams['legend.fontsize'] = 12 
plt.rcParams['figure.titlesize'] = 12 
plt.rcParams['image.cmap'] = 'jet' 
plt.rcParams['image.interpolation'] = 'none' 
plt.rcParams['figure.figsize'] = (16, 8) 
plt.rcParams['lines.linewidth'] = 2 
plt.rcParams['lines.markersize'] = 8
colors = ['xkcd:pale orange', 'xkcd:sea blue', 'xkcd:pale red', 'xkcd:sage green', 'xkcd:terra cotta', 'xkcd:dull purple', 'xkcd:teal', 'xkcd: goldenrod', 'xkcd:cadet blue',
'xkcd:scarlet']
cmap_big = cm.get_cmap('Spectral', 512)
cmap = mcolors.ListedColormap(cmap_big(np.linspace(0.7, 0.95, 256))) 
bbox_props = dict(boxstyle="round,pad=0.3", fc=colors[0], alpha=.5)

In [93]:
dataset=pd.read_csv('arsolarflaresdata.csv').drop(columns=['Unnamed: 0'])


In [94]:
size=250

In [95]:
entries = os.listdir('/Users/pierohmd/Downloads/Solar Flares And Active Regions/')
entries.sort(key=natural_keys)
del entries[entries.index('.ipynb_checkpoints')]
del entries[entries.index('.DS_Store')]
data=[]
IMAGES=[]
for entry in entries:
    try:
        image = Image.open(entry)
        IMAGES.append(image)
        #image=image.convert('rgb')
        new_image = image.resize((size,size))
        data.append(np.array(new_image))
        
    except:
        #print(entry)
        del entries[entries.index(entry)]

data=np.array(data)

In [96]:
del entries[len(entries)-1]

In [97]:
labels=[]
for entry in entries:
    SPLITTED=entry.split('.')
    filename=SPLITTED[0][0]
    if str(filename)=='A':
        labels.append(0)
    else:
        labels.append(1)

In [98]:
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(3, (3, 3), input_shape = (size,size, 1), activation = 'relu'))

classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(3, (3, 3), input_shape = (size,size, 1), activation = 'relu'))

classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())

#classifier.add(Dense(units = 32, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_49 (Conv2D)           (None, 248, 248, 3)       30        
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 124, 124, 3)       0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 122, 122, 3)       84        
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 61, 61, 3)         0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 11163)             0         
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 11164     
Total params: 11,278
Trainable params: 11,278
Non-trainable params: 0
_________________________________________________

In [101]:
train_images, test_images, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.15)
train_images=np.array(train_images)
test_images=np.array(test_images)
train_images, test_images = train_images / 255.0, test_images / 255.0
Train_images=[]
Test_images=[]
for i in range(len(train_images)):
    a=train_images[i].reshape(size,size,1)
    Train_images.append(a)
Train_images=np.array(Train_images)
for j in range(len(test_images)):
    b=test_images[j].reshape(size,size,1)
    Test_images.append(b)
Test_images=np.array(Test_images)
train_images,test_images=Train_images, Test_images
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
history = classifier.fit(train_images, train_labels, epochs=10, 
                validation_data=(test_images, test_labels),batch_size=10)

Train on 111 samples, validate on 20 samples
Epoch 1/10
111/111 [==============================] - 5s 42ms/step - loss: 0.6421 - accuracy: 0.7207 - val_loss: 0.5762 - val_accuracy: 0.7000
Epoch 2/10
111/111 [==============================] - 3s 29ms/step - loss: 0.5478 - accuracy: 0.7207 - val_loss: 0.5051 - val_accuracy: 0.7000
Epoch 3/10
111/111 [==============================] - 4s 32ms/step - loss: 0.4909 - accuracy: 0.7207 - val_loss: 0.4408 - val_accuracy: 0.7000
Epoch 4/10
111/111 [==============================] - 3s 30ms/step - loss: 0.3644 - accuracy: 0.9009 - val_loss: 0.3133 - val_accuracy: 0.9000
Epoch 5/10
111/111 [==============================] - 3s 29ms/step - loss: 0.2407 - accuracy: 0.9640 - val_loss: 0.1937 - val_accuracy: 0.9000
Epoch 6/10
111/111 [==============================] - 3s 30ms/step - loss: 0.1789 - accuracy: 0.9730 - val_loss: 0.1268 - val_accuracy: 0.9000
Epoch 7/10
111/111 [==============================] - 3s 29ms/step - loss: 0.1354 - accuracy: 0.9

In [103]:
ACC=[]
for i in range(10):
    classifier = Sequential()

    # Step 1 - Convolution
    classifier.add(Conv2D(3, (3, 3), input_shape = (size,size, 1), activation = 'relu'))

    classifier.add(MaxPooling2D(pool_size = (2, 2)))
    classifier.add(Conv2D(3, (3, 3), input_shape = (size,size, 1), activation = 'relu'))

    classifier.add(MaxPooling2D(pool_size = (2, 2)))
    classifier.add(Flatten())

    #classifier.add(Dense(units = 32, activation = 'relu'))
    classifier.add(Dense(units = 1, activation = 'sigmoid'))

    # Compiling the CNN
    classifier.summary()
    train_images, test_images, train_labels, test_labels = train_test_split(
        data, labels, test_size=0.15)
    train_images=np.array(train_images)
    test_images=np.array(test_images)
    train_images, test_images = train_images / 255.0, test_images / 255.0
    Train_images=[]
    Test_images=[]
    for i in range(len(train_images)):
        a=train_images[i].reshape(size,size,1)
        Train_images.append(a)
    Train_images=np.array(Train_images)
    for j in range(len(test_images)):
        b=test_images[j].reshape(size,size,1)
        Test_images.append(b)
    Test_images=np.array(Test_images)
    train_images,test_images=Train_images, Test_images
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    history = classifier.fit(train_images, train_labels, epochs=15, 
                    validation_data=(test_images, test_labels),batch_size=10)
    ACC.append(history.history['val_accuracy'][len(history.history['val_accuracy'])-1])

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_53 (Conv2D)           (None, 248, 248, 3)       30        
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 124, 124, 3)       0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 122, 122, 3)       84        
_________________________________________________________________
max_pooling2d_54 (MaxPooling (None, 61, 61, 3)         0         
_________________________________________________________________
flatten_27 (Flatten)         (None, 11163)             0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 11164     
Total params: 11,278
Trainable params: 11,278
Non-trainable params: 0
_________________________________________________

111/111 [==============================] - 4s 32ms/step - loss: 0.0676 - accuracy: 0.9730 - val_loss: 0.0766 - val_accuracy: 0.9500
Epoch 14/15
111/111 [==============================] - 3s 31ms/step - loss: 0.0691 - accuracy: 0.9730 - val_loss: 0.0685 - val_accuracy: 1.0000
Epoch 15/15
111/111 [==============================] - 4s 32ms/step - loss: 0.0591 - accuracy: 0.9820 - val_loss: 0.0827 - val_accuracy: 0.9500
Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_63 (Conv2D)           (None, 248, 248, 3)       30        
_________________________________________________________________
max_pooling2d_63 (MaxPooling (None, 124, 124, 3)       0         
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 122, 122, 3)       84        
_________________________________________________________________
max_pooling2d_64 (MaxPooling 

Train on 111 samples, validate on 20 samples
Epoch 1/15
111/111 [==============================] - 5s 45ms/step - loss: 0.5583 - accuracy: 0.7387 - val_loss: 0.8467 - val_accuracy: 0.5500
Epoch 2/15
111/111 [==============================] - 3s 31ms/step - loss: 0.4553 - accuracy: 0.7658 - val_loss: 0.5721 - val_accuracy: 0.6000
Epoch 3/15
111/111 [==============================] - 3s 31ms/step - loss: 0.3402 - accuracy: 0.8108 - val_loss: 0.5128 - val_accuracy: 0.6500
Epoch 4/15
111/111 [==============================] - 3s 30ms/step - loss: 0.2518 - accuracy: 0.9640 - val_loss: 0.4345 - val_accuracy: 0.9000
Epoch 5/15
111/111 [==============================] - 3s 31ms/step - loss: 0.1786 - accuracy: 0.9459 - val_loss: 0.3477 - val_accuracy: 0.9500
Epoch 6/15
111/111 [==============================] - 3s 31ms/step - loss: 0.1366 - accuracy: 0.9820 - val_loss: 0.3941 - val_accuracy: 0.9500
Epoch 7/15
111/111 [==============================] - 4s 33ms/step - loss: 0.1206 - accuracy: 0.9

In [107]:
print(np.array(ACC).mean())

0.9559999958276748